In [301]:
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup as soup
import time, pickle, ssl

import datetime, requests

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
current_date = datetime.datetime.now()

#### URLs

In [302]:
url_listagem_empresas = "http://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/empresas-listadas.htm"
url_search_empresa = "http://bvmf.bmfbovespa.com.br/pt-br/mercados/acoes/empresas/ExecutaAcaoConsultaInfoEmp.asp?CodCVM={CVM_CODE}&ViewDoc=0#a"
url_historico_demonstrativos = "http://bvmf.bmfbovespa.com.br/cias-listadas/empresas-listadas/HistoricoFormularioReferencia.aspx?codigoCVM={CVM_CODE}&tipo=itr&ano=0&idioma=pt-br"

### Get available tickers

In [303]:
import json
import requests

In [304]:
class B3EmpresasScraper(object):
    def __init__(self):
        self.search_request = {"State":{},
                               "TabState":{"ctl00_contentPlaceHolderConteudo_tabMenuEmpresaListada_tabNome":{"Selected":True}}}
    def scrape(self):
        jobs = self.scrape_jobs()
        for job in jobs:
            print(job)
                
    def scrape_jobs(self, max_pages=3):
        jobs = []
#         pageno = 0
#         self.search_request['pageNumber'] = pageno

#         while pageno < max_pages:
        payload = { 
            '__EVENTTARGET': 'ctl00:contentPlaceHolderConteudo:BuscaNomeEmpresa1:btnTodas',
            '__VIEWSTATE': '/wEPDwULLTE2OTg3NzU5NjgPZBYCZg9kFgQCAQ9kFgICEA8WAh4EVGV4dAUIbWVyY2Fkb3NkAgMPZBYCAgEPZBYEAgEPZBYCAgEPZBYGAgUPDxYCHwAFCjIxLzExLzIwMTlkZAIJDw8WAh8ABQUyM2gwMGRkAhEPDxYCHgVDb3VudAIEZBYCAgMPDxYGHgJJRAUacHZ3QnVzY2FBbm9MaXN0YWdlbUVtcHJlc2EeB1Rvb2xUaXBlHgdWaXNpYmxlaGQWAgIBD2QWAgIBDxBkZBYAZAIDDw9kFgIeB29uY2xpY2sFHGhpc3RvcnkuYmFjaygpOyByZXR1cm4gZmFsc2VkGAYFPGN0bDAwJGNvbnRlbnRQbGFjZUhvbGRlckNvbnRldWRvJEJ1c2NhU2V0b3JFbXByZXNhMSRncmRTZXRvcg8UKwACZBcAZAVBY3RsMDAkY29udGVudFBsYWNlSG9sZGVyQ29udGV1ZG8kQnVzY2FTZWdtZW50b0VtcHJlc2ExJGdyZEVtcHJlc2EPFCsAAmQXAGQFHl9fQ29udHJvbHNSZXF1aXJlUG9zdEJhY2tLZXlfXxYIBT1jdGwwMCRjb250ZW50UGxhY2VIb2xkZXJDb250ZXVkbyRCdXNjYU5vbWVFbXByZXNhMSRncmRFbXByZXNhBTxjdGwwMCRjb250ZW50UGxhY2VIb2xkZXJDb250ZXVkbyRCdXNjYVNldG9yRW1wcmVzYTEkZ3JkU2V0b3IFPmN0bDAwJGNvbnRlbnRQbGFjZUhvbGRlckNvbnRldWRvJEJ1c2NhU2V0b3JFbXByZXNhMSRncmRFbXByZXNhBUFjdGwwMCRjb250ZW50UGxhY2VIb2xkZXJDb250ZXVkbyRCdXNjYVNlZ21lbnRvRW1wcmVzYTEkZ3JkRW1wcmVzYQVEY3RsMDAkY29udGVudFBsYWNlSG9sZGVyQ29udGV1ZG8kQnVzY2FBbm9MaXN0YWdlbUVtcHJlc2ExJGdyZEVtcHJlc2EFL2N0bDAwJGNvbnRlbnRQbGFjZUhvbGRlckNvbnRldWRvJEFqYXhQYW5lbEJ1c2NhBStjdGwwMCRjb250ZW50UGxhY2VIb2xkZXJDb250ZXVkbyRtcGdQYWdpbmFzBTZjdGwwMCRjb250ZW50UGxhY2VIb2xkZXJDb250ZXVkbyR0YWJNZW51RW1wcmVzYUxpc3RhZGEFRGN0bDAwJGNvbnRlbnRQbGFjZUhvbGRlckNvbnRldWRvJEJ1c2NhQW5vTGlzdGFnZW1FbXByZXNhMSRncmRFbXByZXNhDxQrAAJkFwBkBT5jdGwwMCRjb250ZW50UGxhY2VIb2xkZXJDb250ZXVkbyRCdXNjYVNldG9yRW1wcmVzYTEkZ3JkRW1wcmVzYQ8UKwACZBcAZAU9Y3RsMDAkY29udGVudFBsYWNlSG9sZGVyQ29udGV1ZG8kQnVzY2FOb21lRW1wcmVzYTEkZ3JkRW1wcmVzYQ8UKwACZBcAZILXCPCm5ktP5zeUM9V5d8CeqVJU',
            '__VIEWSTATEGENERATOR': '9CFBB4E5',
            '__EVENTVALIDATION': '/wEWKQKQw/XtAQKrqYeWCwL85ODVCQLh+8LADwKyt5yADgKXzv7qAwLoidiqAgLNoLqVCALuyLaBBALT35jsCQKkm/KrCAKJstSWDgLa7a3WDAK/hJDBAgKQwOmAAQL11svrBgL2pYL/BwLbvOTpDQKs+L2pDAKRj6CUAgLiyvlTAsfh274GApidtf4EAv2zl+kKAp7ck9UGAoPz9b8MAtSuz/8KApb/x9cCAvuVqsIIAszRg4IHArHo5ewMAoKkv6wLAue6oZcBArj2+tYPAp2N3cEFAr612a0BAqPMu5gHAqqynsAFAvrzvPwMAr/S5NkCArSz5PIOQBNPwVxP46TJ4TGUiUDReAcNROw=',
            'ctl00_contentPlaceHolderConteudo_AjaxPanelBuscaPostDataValue': 'ctl00_contentPlaceHolderConteudo_AjaxPanelBusca,ActiveElement,ctl00_contentPlaceHolderConteudo_BuscaNomeEmpresa1_btnTodas;',
            'ctl00$contentPlaceHolderConteudo$tabMenuEmpresaListada': json.dumps(self.search_request),
            'ctl00$contentPlaceHolderConteudo$mpgPaginas_Selected' : '0',
            'RadAJAXControlID': 'ctl00_contentPlaceHolderConteudo_AjaxPanelBusca',
            'httprequest': 'true'
        }

        r = requests.post(
            url='http://bvmf.bmfbovespa.com.br/cias-listadas/empresas-listadas/BuscaEmpresaListada.aspx?idioma=pt-br',
            data=payload,
            headers={
                'X-Requested-With': 'HttpRequest'
            }
        )

#         s = BeautifulSoup(r.text)
#             if not s.requisition:
#                 break

#             for r in s.findAll('ctl00_contentPlaceHolderConteudo_tabMenuEmpresaListada_tabNome'):
#                 job = {}
#                 job['jobid'] = r.jobid.text
#                 job['title'] = r.postingtitle and \
#                     r.postingtitle.text or r.retailpostingtitle.text
#                 job['location'] = r.location.text
#                 jobs.append(job)

#             # Next page
#             pageno += 1
#             self.search_request['pageNumber'] = pageno

        return r

In [305]:
scraper = B3EmpresasScraper()
pg = scraper.scrape_jobs()
raw_soup = soup(pg.text, "html.parser")
raw_companies = raw_soup.find('tbody').findAll('tr')

In [306]:
empresas_listadas = {cvm_code:{'name':n,'segment':s} for (cvm_code,n,s) in zip ([int(c.findAll('a')[0]['href'].replace('ResumoEmpresaPrincipal.aspx?codigoCvm=','')) for c in raw_companies],
                                                                  [c.findAll('a')[0].text.strip() for c in raw_companies],
                                                                  [c.findAll('td')[2].text.strip() for c in raw_companies])}

In [307]:
# url_historico_demonstrativos.format(CVM_CODE=19348)

In [308]:
# req = Request(url=url_historico_demonstrativos.format(CVM_CODE=19348))
# uClient = urlopen(req)
# page_html = uClient.read()
# uClient.close()

In [309]:
# raw_soup = soup(page_html, "html.parser")
# raw_soup

In [310]:
# raw_soup.find('div',{'id':'ctl00_contentPlaceHolderConteudo_divDemonstrativo'}).find('div',{'class':'list-avatar'}).findAll('a')

In [311]:
# https://www.rad.cvm.gov.br/ENETCONSULTA/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=88999&CodigoTipoInstituicao=2
    
# http://www.rad.cvm.gov.br/ENETCONSULTA/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=88999&amp;CodigoTipoInstituicao=2

In [312]:
# url_found = 'http://www.rad.cvm.gov.br/ENETCONSULTA/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=88999&amp;CodigoTipoInstituicao=2'
# url_found = url_found.replace('http','https').replace('&amp;','&')
# url_found

In [313]:
# json_post = {'codigoInstituicao': '1', 'numeroProtocolo': '88999'}

In [314]:
# import ssl

# gcontext = ssl.SSLContext()

# req = Request(url=url_found,headers=headers)
# uClient = urlopen(req, context=gcontext)
# page_html = uClient.read()
# uClient.close()
# raw_soup = soup(page_html, "html.parser")
# raw_soup

In [315]:
# raw_soup.find('table',{'id':'ctl00_cphPopUp_tbDados'})

In [316]:
# [(k,empresas_listadas[k]) for k in empresas_listadas.keys() if 'ita' in empresas_listadas[k]['name'].lower()]

## AQUI TÁ O ENDEREÇO DA TABELA! SUBSTITUIR REGISTRO CVM, NUMERO SEQUENCIAL DOCUMENTO E DATA REFERENCIA

In [317]:
url_site_novo = "https://www.rad.cvm.gov.br/ENETCONSULTA/frmDemonstracaoFinanceiraITR.aspx?Informacao=2&Demonstracao=4&Periodo=0&Grupo=&Quadro=&NomeTipoDocumento=&Empresa=&DataReferencia=2019-09-30&Versao=1&CodTipoDocumento=3&NumeroSequencialDocumento=88999&NumeroSequencialRegistroCvm=1865&CodigoTipoInstituicao=2"
url_site_novo

'https://www.rad.cvm.gov.br/ENETCONSULTA/frmDemonstracaoFinanceiraITR.aspx?Informacao=2&Demonstracao=4&Periodo=0&Grupo=&Quadro=&NomeTipoDocumento=&Empresa=&DataReferencia=2019-09-30&Versao=1&CodTipoDocumento=3&NumeroSequencialDocumento=88999&NumeroSequencialRegistroCvm=1865&CodigoTipoInstituicao=2'

In [318]:
# https://www.rad.cvm.gov.br/ENETCONSULTA/frmDemonstracaoFinanceiraITR.aspx?Informacao=2&Demonstracao=4&Periodo=0&Grupo=&Quadro=&NomeTipoDocumento=&Empresa=&DataReferencia=2019-09-30&Versao=1&CodTipoDocumento=3&NumeroSequencialDocumento=88999&NumeroSequencialRegistroCvm=1865&CodigoTipoInstituicao=2

In [319]:
codigos_tipo_documento = {1:'Balanço Patrimonial Ativo',
                          2:'Balanço Patrimonial Passivo',
                          3: 'Demonstração do Resultado',
                          4: 'Demonstração do Resultado Abrangente',
                          5: 'Demonstração do Fluxo de Caixa',
                          6: 'Demosntração das Mutações de Patr. Líq.',
                          7: 'Demosntração de Valor Adicionado'}

In [320]:
url_lista_hist = "http://bvmf.bmfbovespa.com.br/cias-listadas/empresas-listadas/HistoricoFormularioReferencia.aspx?codigoCVM={CVM_CODE}&tipo=itr&ano=0&idioma=pt-br"
url_lista_hist.format(CVM_CODE=19348)

'http://bvmf.bmfbovespa.com.br/cias-listadas/empresas-listadas/HistoricoFormularioReferencia.aspx?codigoCVM=19348&tipo=itr&ano=0&idioma=pt-br'

In [321]:
gcontext = ssl.SSLContext()

req = Request(url=url_lista_hist.format(CVM_CODE=19348),headers=headers)
uClient = urlopen(req, context=gcontext)
page_html = uClient.read()
uClient.close()
raw_soup = soup(page_html, "html.parser")
raw_soup


<!DOCTYPE doctype html>

<html class="no-js" lang="pt-br">
<head id="ctl00_Head1"><link href="../../aspnet_client/system_web/2_0_50727/Themes/SiteBmfBovespa/Calendar/Skins/SiteBmfBovespa/Calendar.css" rel="stylesheet" type="text/css"/><link href="../../aspnet_client/system_web/2_0_50727/Themes/SiteBmfBovespa/Calendar/Skins/SiteBmfBovespa/RadTimePicker.css" rel="stylesheet" type="text/css"/><link href="../../aspnet_client/system_web/2_0_50727/Themes/SiteBmfBovespa/Combobox/Skins/SiteBmfBovespa/Styles.css" rel="stylesheet" type="text/css"/><link href="../../aspnet_client/system_web/2_0_50727/Themes/SiteBmfBovespa/Estrutura.css" rel="stylesheet" type="text/css"/><link href="../../aspnet_client/system_web/2_0_50727/Themes/SiteBmfBovespa/foundation.css" rel="stylesheet" type="text/css"/><link href="../../aspnet_client/system_web/2_0_50727/Themes/SiteBmfBovespa/Grid/Skins/SiteBmfBovespa/Styles.css" rel="stylesheet" type="text/css"/><link href="../../aspnet_client/system_web/2_0_50727/Themes

In [322]:
def datToAnomesdia(d):
    return int(d[6:10])*10000+int(d[3:5])*100+int(d[0:2])
def anomesdiaToDat(d):
    return "%02d"%(d%100)+"/"+"%02d"%((d//100)%100)+"/"+"%04d"%(d//10000)
def datToUrlDat(d):
    dat = d.split('/')
    return dat[2]+"-"+dat[1]+"-"+dat[0]

In [323]:
datas_site_novo[3].split('/')

['30', '09', '2018']

In [324]:
relatorios_raw = raw_soup.find('div',{'class':'list-avatar'}).findAll('p')
poss_relat = [k.text.replace('\n','') for k in relatorios_raw]
poss_relat

['30/09/2019 - Informações Trimestrais - Versão 1.0',
 '30/06/2019 - Informações Trimestrais - Versão 2.0',
 '30/06/2019 - Informações Trimestrais - Versão 1.0',
 '31/03/2019 - Informações Trimestrais - Versão 1.0',
 '30/09/2018 - Informações Trimestrais - Versão 1.0',
 '30/06/2018 - Informações Trimestrais - Versão 1.0',
 '31/03/2018 - Informações Trimestrais - Versão 1.0',
 '30/09/2017 - Informações Trimestrais - Versão 1.0',
 '30/06/2017 - Informações Trimestrais - Versão 2.0',
 '30/06/2017 - Informações Trimestrais - Versão 1.0',
 '31/03/2017 - Informações Trimestrais - Versão 1.0',
 '30/09/2016 - Informações Trimestrais - Versão 1.0',
 '30/06/2016 - Informações Trimestrais - Versão 2.0',
 '30/06/2016 - Informações Trimestrais - Versão 1.0',
 '31/03/2016 - Informações Trimestrais - Versão 1.0',
 '30/09/2015 - Informações Trimestrais - Versão 1.0',
 '30/06/2015 - Informações Trimestrais - Versão 1.0',
 '31/03/2015 - Informações Trimestrais - Versão 1.0',
 '30/09/2014 - Informações T

In [325]:
# [k.text.replace('\n','') for (k,amd) in zip(raw_soup.find('div',{'class':'list-avatar'}).findAll('p'),
#                                             [datToAnomesdia(k.text.replace('\n','')) for k in raw_soup.find('div',{'class':'list-avatar'}).findAll('p')]) \
#  if (anomesdiaToDat(amd) in [anomesdiaToDat(d) for (d,c) in zip(dat,count) if c > 1])]

In [326]:
(dat,count) = np.unique([datToAnomesdia(k.text.replace('\n','')) for k in raw_soup.find('div',{'class':'list-avatar'}).findAll('p')],return_counts=True)

idxs_se_rep = [list([txt.text for txt in raw_soup.find('div',{'class':'list-avatar'}).findAll('p')]).index(k.text) for (k,amd) in zip(raw_soup.find('div',{'class':'list-avatar'}).findAll('p'),
                                            [datToAnomesdia(k.text.replace('\n','')) for k in raw_soup.find('div',{'class':'list-avatar'}).findAll('p')]) \
 if (anomesdiaToDat(amd) in [anomesdiaToDat(d) for (d,c) in zip(dat,count) if c > 1])&(('2.0' in k.text.replace('\n',''))|('Reapr' in k.text.replace('\n','')))]

idxs_n_se_rep = [list([txt.text for txt in raw_soup.find('div',{'class':'list-avatar'}).findAll('p')]).index(k.text) for (k,amd) in zip(raw_soup.find('div',{'class':'list-avatar'}).findAll('p'),
                                            [datToAnomesdia(k.text.replace('\n','')) for k in raw_soup.find('div',{'class':'list-avatar'}).findAll('p')]) \
 if (anomesdiaToDat(amd) in [anomesdiaToDat(d) for (d,c) in zip(dat,count) if c < 2])]

idxs_to_get = list(set(idxs_se_rep)|set(idxs_n_se_rep))

In [327]:
# nums_docs = [int(r.a['href'].split('NumeroSequencialDocumento=')[1].split('&')[0]) for r in [relatorios_raw[i] for i in idxs_to_get]]

In [328]:
datas_site_antigo = [poss_relat[i].split(' ')[0] for i in idxs_to_get if "www2.bmfbovespa.com.br" in relatorios_raw[i].a['href']]
datas_site_antigo

['30/09/2010',
 '30/06/2010',
 '31/03/2010',
 '30/09/2009',
 '30/06/2009',
 '31/03/2009',
 '30/09/2008',
 '30/06/2008',
 '31/03/2008',
 '30/09/2007',
 '30/06/2007',
 '31/03/2007',
 '30/09/2006',
 '30/06/2006',
 '31/03/2006',
 '30/09/2005',
 '30/06/2005',
 '31/03/2005',
 '30/09/2004',
 '30/06/2004',
 '31/03/2004',
 '30/09/2003',
 '30/06/2003',
 '31/03/2003',
 '30/09/2002',
 '30/06/2002',
 '31/03/2002',
 '30/09/2001',
 '30/06/2001',
 '31/03/2001',
 '30/09/2000',
 '30/06/2000',
 '31/03/2000',
 '30/09/1999',
 '30/06/1999',
 '31/03/1999',
 '30/09/1998']

In [329]:
count = -1
for i in [relatorios_raw[i] for i in idxs_to_get]:
    count+=1
    try:
        if "www.rad.cvm.gov.br" in i.a['href']:
            print(int(i.a['href'].split('NumeroSequencialDocumento=')[1].split('&')[0]))
        elif "www2.bmfbovespa.com.br" in i.a['href']:
            print('site antigo')
    except:
        print(count)

88999
89812
82594
78354
76576
73624
69351
67924
64595
60256
58400
55455
51223
49416
46366
41985
40111
36464
32096
30055
26419
21825
19848
16379
12279
10497
7014
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo
site antigo


In [330]:
r = [relatorios_raw[i] for i in idxs_to_get][26]
int(r.a['href'].split('NumeroSequencialDocumento=')[1].split('&')[0])

7014

### Para site "antigo":

In [331]:
url_open_site_antigo = "http://www2.bmfbovespa.com.br/dxw/FrDXW.asp?site=B&mercado=16&razao=ITAU%20UNIBANCO%20HOLDING%20S.A.&pregao=ITAUUNIBANCO&ccvm=19348&data={DATA_INPUT}&tipo=4"

url_relatorios_site_antigo = {'ativo':'http://www2.bmfbovespa.com.br/dxw/FormDetalheDXWBalanco.asp?TipoInfo=T&Tipo=01%20-%20Ativo',
                              'passivo':'http://www2.bmfbovespa.com.br/dxw/FormDetalheDXWBalanco.asp?TipoInfo=T&Tipo=02%20-%20Passivo',
                              'dre':'http://www2.bmfbovespa.com.br/dxw/FormDetalheDXWDRE.asp?TipoInfo=T',
                             }

In [332]:
# s = requests.Session()
# url = url_teste_0
# r = s.get(url=url_teste_0)
# Logincookies = r.cookies
# r = s.get(url=url_teste_1,cookies=Logincookies)
# raw_soup = soup(r.text,"html.parser")
# raw_soup

In [333]:
with requests.Session() as s:
    r = s.get(url=url_open_site_antigo.format(DATA_INPUT=datas_site_antigo[0]))
    Logincookies = r.cookies
    
    r = s.get(url=url_relatorios_site_antigo['ativo'],cookies=Logincookies)
    raw_soup = soup(r.text,"html.parser")
raw_soup


<html>
<head>
<meta content="no-cache" http-equiv="Cache Control"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="-1" http-equiv="Expires"/>
<title>Grupo 08 - Balanço Patrimonial Consolidado - </title>
<script data-dtconfig="rid=RID_446613157|rpid=909272051|domain=bmfbovespa.com.br|reportUrl=/DXW/rb_8370fec7-c82e-413f-a2c6-777046ed9811|app=5286dfffe4e737f8|featureHash=ICA27QSVdefgijmoqruvx|msl=153600|srsr=10000|rdnt=1|uxrgce=1|bp=3|srms=1,1,,,|uxrgcm=100,25,300,3;100,25,300,3|dpvc=1|lastModification=1576180310097|dtVersion=10179191120132458|tp=500,50,0,1|uxdcw=1500|agentUri=/DXW/ruxitagentjs_ICA27QSVdefgijmoqruvx_10179191120132458.js" src="/DXW/ruxitagentjs_ICA27QSVdefgijmoqruvx_10179191120132458.js" type="text/javascript"></script><link href="./Library/LayoutDXW.css" rel="stylesheet" type="text/css"/>
</head>
<body alink="#CC0000" background="./imagem/fundo_destaques.gif" bgcolor="#FFFFFF" bgproperties="fixed" link="#000000" scroll="auto" text="#000000" vlink="#6A6A6A

In [334]:
table_chunks = raw_soup.find('div',{'class':'ScrollMaker'}).findAll('tr')
df = pd.DataFrame([[str.strip(k.text) for k in n.findAll('td')] for n in table_chunks])
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0)).reset_index(drop=True)
df.columns.name = None
for col in df.columns[-2:]:
    df[col] = [int(k.replace('.','')) if '(' not in k else \
               (-1)*int(k.replace('.','').replace('(','').replace(')','')) \
               for k in df[col]]
df    

,Código da Conta,Descrição da Conta,30/09/2010,30/06/2010
0,1,Ativo Total,686248444,651582990
1,1.01,Ativo Circulante,502876792,494058520
2,1.01.01,Disponibilidades,11063416,12415203
3,1.01.02,Aplicações Interfinanceiras de Liquidez,111816439,114638668
4,1.01.02.01,Aplicações no Mercado Aberto,95757805,98319115
5,1.01.02.02,Aplicações no Mercado Aberto - Rec.Garan,3397239,3431152
6,1.01.02.03,Aplicações em Depósitos Interfinanceiros,12661395,12888401
7,1.01.03,Títulos e Valores Mobiliários,110184583,105558185
8,1.01.03.01,Carteira Própria,32120828,31227405
9,1.01.03.02,Vinculados a Compromisso de Recompra,10578242,14571187


In [335]:
for col in df.columns[-2:]:
    print(col)

30/09/2010
30/06/2010


#### Pegando todos os relatórios de uma vez (site antigo):

In [336]:
relatorios_site_antigo = {}
for dat in datas_site_antigo:
    with requests.Session() as s:
        r = s.get(url=url_open_site_antigo.format(DATA_INPUT=dat))
        Logincookies = r.cookies
        relatorios_site_antigo[dat] = {}
        for k in url_relatorios_site_antigo.keys():  
            r = s.get(url=url_relatorios_site_antigo[k],cookies=Logincookies)
            raw_soup = soup(r.text,"html.parser")
            try:
                table_chunks = raw_soup.find('div',{'class':'ScrollMaker'}).findAll('tr')   
                if k in ['ativo','passivo']:
                    df = pd.DataFrame([[str.strip(k.text) for k in n.findAll('td')] for n in table_chunks])
                    df.columns = df.iloc[0]
                    df = df.reindex(df.index.drop(0)).reset_index(drop=True)
                    df.columns.name = None
                    for col in df.columns[-2:]:
                        df[col] = [int(k.replace('.','')) if '(' not in k else \
                                   (-1)*int(k.replace('.','').replace('(','').replace(')','')) \
                                   for k in df[col]]
                else:
                    df = pd.DataFrame([[str.strip(k.text) for k in n.findAll('td')] for n in table_chunks[0].findAll('tr')],\
                                      columns = [k.text for k in table_chunks[0].findAll('td')[0:6]])
                    for col in df.columns[-4:]:
                        df[col] = [int(k.replace('.','')) if '(' not in k else \
                                   (-1)*int(k.replace('.','').replace('(','').replace(')','')) \
                                   for k in df[col]]
                relatorios_site_antigo[dat][k] = df
            except:
                continue
        print("\r%.1f%% concluído"%((datas_site_antigo.index(dat)+1)*100.0/len(datas_site_antigo)),end='')
for k in [k for k in relatorios_site_antigo.keys() if relatorios_site_antigo[k] == {}]:
    del relatorios_site_antigo[k]

100.0% concluído

In [337]:
relatorios_site_antigo['30/09/2010']['dre']

,Código da Conta,Descrição da Conta,Valor do Trimestre Atual 01/07/2010 a 30/09/2010,Valor Acumulado do Atual Exercício 01/01/2010 a 30/09/2010,Valor do Igual Trimestre do Exercício Anterior 01/07/2009 a 30/09/2009,Valor Acumulado do Exercício Anterior 01/01/2009 a 30/09/2009
0,3.01,Receitas da Intermediação Financeira,21128127,58399028,18928675,58331333
1,3.01.01,Operações de Crédito,12997991,37905017,11736263,35434728
2,3.01.03,Resultado Títulos e Valores Mobiliário,5324423,13697814,5878931,18929553
3,3.01.04,Receita Financeiro Oper.Seg.Prev.Cap.,1576971,3345086,1215451,3549287
4,3.01.05,Resultado de Operações de Câmbio,-121284,833829,-74044,-94334
5,3.01.06,Resultado das Aplicações Compulsórias,1350026,2617282,172074,512099
6,3.02,Despesas da Intermediação Financeira,-12090728,-34455852,-10809467,-34119250
7,3.02.01,Operações de Captação no Mercado Aberto,-8078566,-21916832,-5992171,-20176640
8,3.02.02,Despesas Financeiras Prov.Tec.Prev.e Cap,-1372866,-2898334,-1015743,-3089433
9,3.02.03,Operações de Empréstimos e Repasses,295602,-632542,-74424,89159


### Para site Novo

In [338]:
codigos_tipo_documento = {2:'Balanço Patrimonial Ativo',
                          3:'Balanço Patrimonial Passivo',
                          4: 'Demonstração do Resultado',
#                           4: 'Demonstração do Resultado Abrangente',
#                           5: 'Demonstração do Fluxo de Caixa',
#                           6: 'Demosntração das Mutações de Patr. Líq.',
#                           7: 'Demosntração de Valor Adicionado'}

SyntaxError: unexpected EOF while parsing (<ipython-input-338-92f487194f9d>, line 7)

In [341]:
nums_seq_doc_site_novo = [int(relatorios_raw[i].a['href'].split('NumeroSequencialDocumento=')[1].split("&")[0]) for i in idxs_to_get if "www.rad.cvm.gov.br" in relatorios_raw[i].a['href']]
datas_site_novo = [poss_relat[i].split(' ')[0] for i in idxs_to_get if "www.rad.cvm.gov.br" in relatorios_raw[i].a['href']]
# datas_site_novo

In [ ]:
18970

In [350]:
url_site_novo = "https://www.rad.cvm.gov.br/ENETCONSULTA/frmDemonstracaoFinanceiraITR.aspx?Informacao=2&Demonstracao={TIPO_DOC}&Periodo=0&Grupo=&Quadro=&NomeTipoDocumento=&Empresa=&DataReferencia={DAT_REF}&Versao=1&CodTipoDocumento=3&NumeroSequencialDocumento={NUM_SEQ_DOC}&NumeroSequencialRegistroCvm={COD_CVM}&CodigoTipoInstituicao=2"
url_site_novo.format(TIPO_DOC=3,DAT_REF=datToUrlDat(datas_site_novo[0]),NUM_SEQ_DOC=nums_seq_doc_site_novo[0],COD_CVM=19348)

'https://www.rad.cvm.gov.br/ENETCONSULTA/frmDemonstracaoFinanceiraITR.aspx?Informacao=2&Demonstracao=3&Periodo=0&Grupo=&Quadro=&NomeTipoDocumento=&Empresa=&DataReferencia=2019-09-30&Versao=1&CodTipoDocumento=3&NumeroSequencialDocumento=88999&NumeroSequencialRegistroCvm=19348&CodigoTipoInstituicao=2'

In [353]:
"https://www.rad.cvm.gov.br/ENETCONSULTA/frmDemonstracaoFinanceiraITR.aspx?Informacao=2&Demonstracao=3&Periodo=0&Grupo=&Quadro=&NomeTipoDocumento=&Empresa=&DataReferencia=2019-09-30&Versao=1&CodTipoDocumento=3&NumeroSequencialDocumento=88999&NumeroSequencialRegistroCvm=1865&CodigoTipoInstituicao=2"

'https://www.rad.cvm.gov.br/ENETCONSULTA/frmDemonstracaoFinanceiraITR.aspx?Informacao=2&Demonstracao=3&Periodo=0&Grupo=&Quadro=&NomeTipoDocumento=&Empresa=&DataReferencia=2019-09-30&Versao=1&CodTipoDocumento=3&NumeroSequencialDocumento=88999&NumeroSequencialRegistroCvm=1865&CodigoTipoInstituicao=2'

In [361]:
ccvm = 19348
relatorios_site_novo = {}
for (dat,nseq) in zip(datas_site_novo[0:1],nums_seq_doc_site_novo[0:1]):

    gcontext = ssl.SSLContext()
    r_bpa = Request(url=url_site_novo.format(TIPO_DOC=2,DAT_REF=datToUrlDat(dat),NUM_SEQ_DOC=nseq,COD_CVM=ccvm),headers=headers)
    r_bpp = Request(url=url_site_novo.format(TIPO_DOC=3,DAT_REF=datToUrlDat(dat),NUM_SEQ_DOC=nseq,COD_CVM=ccvm),headers=headers)
    r_dre = Request(url=url_site_novo.format(TIPO_DOC=4,DAT_REF=datToUrlDat(dat),NUM_SEQ_DOC=nseq,COD_CVM=ccvm),headers=headers)
    uClient_bpa = urlopen(r_bpa, context=gcontext)
    html_bpa = uClient_bpa.read()
    uClient_bpp = urlopen(r_bpa, context=gcontext)
    html_bpp = uClient_bpp.read()
    uClient_dre = urlopen(r_bpa, context=gcontext)
    html_dre = uClient_dre.read()
    uClient_bpa.close()
    uClient_bpp.close()
    uClient_dre.close()

    raw_soup_bpa = soup(html_bpa, "html.parser")
    raw_soup_bpp = soup(html_bpa, "html.parser")
    raw_soup_dre = soup(html_bpa, "html.parser")

In [356]:
for k in empresas_listadas.keys():
    if 'ita' in empresas_listadas[k]['name'].lower():
        print(k,empresas_listadas[k]['name'])

21725 ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.
20990 B2W - COMPANHIA DIGITAL
17493 CAPITALPART PARTICIPACOES S.A.
3298 CIA HABITASUL DE PARTICIPACOES
19364 ITAPEBI GERACAO DE ENERGIA S.A.
19348 ITAU UNIBANCO HOLDING S.A.
7617 ITAUSA INVESTIMENTOS ITAU S.A.
22160 POLO CAPITAL SECURITIZADORA S.A
18406 RB CAPITAL COMPANHIA DE SECURITIZAÇÃO
19860 RB CAPITAL SECURITIZADORA S.A.


In [378]:
url_request_post_site_novo = "https://www.rad.cvm.gov.br/ENETCONSULTA/frmGerenciaPaginaFRE.aspx/MontarDadosCabecalhoTela"
url_teste = url_site_novo.format(TIPO_DOC=3,DAT_REF=datToUrlDat(datas_site_novo[0]),NUM_SEQ_DOC=nums_seq_doc_site_novo[0],COD_CVM=19348)

In [390]:
payload = { 'codigoInstituicao': '1', 'numeroProtocolo': '88999'}
with requests.Session() as s:
    r = s.get(url="http://www.rad.cvm.gov.br/ENETCONSULTA/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=88999&CodigoTipoInstituicao=2",verify=False)
    Logincookies = r.cookies
    
    r = s.post(url="http://www.rad.cvm.gov.br/ENETCONSULTA/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=88999&CodigoTipoInstituicao=2",cookies=Logincookies,data=payload)
    raw_soup = soup(r.text,"html.parser")
raw_soup

D:\Users\johnm\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
D:\Users\johnm\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



<html xmlns="http://www.w3.org/1999/xhtml">
<head id="Head1"><link href="App_Themes/CVM/Calendar/Skins/CVM/Calendar.css" rel="stylesheet" type="text/css"/><link href="App_Themes/CVM/Calendar/Skins/CVM/RadTimePicker.css" rel="stylesheet" type="text/css"/><link href="App_Themes/CVM/Calendario.css" rel="stylesheet" type="text/css"/><link href="App_Themes/CVM/ComboBox/CVM/Styles.css" rel="stylesheet" type="text/css"/><link href="App_Themes/CVM/ComboBox/Skins/CVM/Styles.css" rel="stylesheet" type="text/css"/><link href="App_Themes/CVM/EditarConteudo.css" rel="stylesheet" type="text/css"/><link href="App_Themes/CVM/Entrada.css" rel="stylesheet" type="text/css"/><link href="App_Themes/CVM/Erro.css" rel="stylesheet" type="text/css"/><link href="App_Themes/CVM/Estrutura.css" rel="stylesheet" type="text/css"/><link href="App_Themes/CVM/Ferramentas.css" rel="stylesheet" type="text/css"/><link href="App_Themes/CVM/Formulario.css" rel="stylesheet" type="text/css"/><link href="App_Themes/CVM/Formul

In [368]:
raw_soup


<html>
<head>
<meta content="no-cache" http-equiv="Cache Control"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="-1" http-equiv="Expires"/>
<title>01 - Demonstração do Resultado</title>
<script data-dtconfig="rid=RID_1010267258|rpid=-1252237128|domain=bmfbovespa.com.br|reportUrl=/DXW/rb_8370fec7-c82e-413f-a2c6-777046ed9811|app=5286dfffe4e737f8|featureHash=ICA27QSVdefgijmoqruvx|msl=153600|srsr=10000|rdnt=1|uxrgce=1|bp=3|srms=1,1,,,|uxrgcm=100,25,300,3;100,25,300,3|dpvc=1|lastModification=1576180310097|dtVersion=10179191120132458|tp=500,50,0,1|uxdcw=1500|agentUri=/DXW/ruxitagentjs_ICA27QSVdefgijmoqruvx_10179191120132458.js" src="/DXW/ruxitagentjs_ICA27QSVdefgijmoqruvx_10179191120132458.js" type="text/javascript"></script><link href="./Library/LayoutDXW.css" rel="stylesheet" type="text/css"/>
</head>
<body alink="#CC0000" background="./imagem/fundo_destaques.gif" bgcolor="#FFFFFF" bgproperties="fixed" link="#000000" scroll="auto" text="#000000" vlink="#6A6A6A">
<center>


#### Interpretando XML baixado do site B3

In [2]:
from zipfile import ZipFile
import pandas as pd
from bs4 import BeautifulSoup as soup

In [62]:
# Nome do arquivo baixado
arquivo = 'petroleo_brasileiro_2019_09_30.zip'

In [64]:
# Só pra ver: arquivos .itr no arquivo baixado
[k for k in ZipFile("testes/"+arquivo, 'r').namelist() if 'itr' in k]

['009512201909300301.itr']

In [66]:
# Extrai arquivo itr para pasta temporária
ZipFile("testes/"+arquivo, 'r').extract([k for k in ZipFile("testes/"+arquivo, 'r').namelist() if 'itr' in k][0], 'testes')

'testes\\009512201909300301.itr'

In [67]:
# Só pra ver: arquivos dentro do arquivo itr
ZipFile("testes/"+[k for k in ZipFile("testes/"+arquivo, 'r').namelist() if 'itr' in k][0], 'r').namelist()

['AnexoDocumento.xml',
 'AnexoTexto.xml',
 'ComposicaoCapitalSocialDemonstracaoFinanceiraNegocios.xml',
 'Documento.xml',
 'FormularioDemonstracaoFinanceiraITR.xml',
 'InfoFinaDFin.xml',
 'InformacaoFinanceiraDemonstracaoFinanceira.xml',
 'JustificativaFaltaDocumento.xml',
 'PagamentoProventoDinheiroDemonstracaoFinanceiraNegocios.xml',
 'PeriodoDemonstracaoFinanceira.xml',
 'ResponsavelEsclarecimentoDuvidas.xml',
 'ResumoInformacaoFinanceiraDemonstracaoFinanceira.xml']

In [68]:
# Extrai InfoFinaDFin pra pasta temporaria
ZipFile("testes/"+[k for k in ZipFile("testes/"+arquivo, 'r').namelist() if 'itr' in k][0], 'r').extract('InfoFinaDFin.xml', 'testes')

'testes\\InfoFinaDFin.xml'

In [84]:
soup_xml = soup(open('testes/InfoFinaDFin.xml','r',encoding='utf-8'))

D:\Users\johnm\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file D:\Users\johnm\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [135]:
tiposDemonstracao = {2:'Balanço Patrimonial Ativo',\
                     3:'Balanço Patrimonial Passivo',\
                     4:'DRE',\
                     5:'DRE Abrangente',\
                     7:'Demonstração do Fluxo de Caixa',\
                     8:'Demonstração de Mutações do Patrimônio Líquido',\
                     9:'Demonstração de Valor Adicionado'
                    }
tiposInformacao = {1:'Individual',
                   2:'Consolidada'
                  }

In [164]:
infos = {'Tipo Demonstração':[int(k.text) for k in soup_xml.findAll('codigotipodemonstracaofinanceira')], \
         'Tipo Informação':[int(k.text) for k in soup_xml.findAll('codigotipoinformacaofinanceira')], \
         'Período':[int(k.text) for k in soup_xml.findAll('numeroidentificacaoperiodo')], \
#          'Num ordem conta': [int(k.text) for k in soup_xml.findAll('numeroordemconta')], \
         'Codigo Conta':[k.text for k in soup_xml.findAll('numeroconta')], \
         'Descr Conta':[k.text for k in soup_xml.findAll('descricaoconta1')]}
for i in range(1,13):
    infos['valConta%d'%i] = [float(k.text) for k in soup_xml.findAll('valorconta%d'%i)]
dados_brutos = pd.DataFrame(infos)
mx_c_c = max([len(k) for k in dados_brutos['Codigo Conta']])
dados_brutos['Codigo Conta Concat'] = [int(k.replace('.',''))*pow(10,(mx_c_c-len(k.replace('.','')))) for k in dados_brutos['Codigo Conta']]
dados_brutos = dados_brutos.sort_values(by=['Tipo Demonstração',\
                                            'Tipo Informação',\
                                            'Período',\
                                            'Codigo Conta Concat'
                                           ]).drop(['Codigo Conta Concat'],axis=1).reset_index(drop=True)

In [165]:
# dados_brutos[(dados_brutos['Tipo Demonstração']==2)&(dados_brutos['Tipo Informação']==2)].sort_values(by='Num ordem conta')
dados_brutos[(dados_brutos['Tipo Demonstração']==2)&(dados_brutos['Tipo Informação']==2)]

,Tipo Demonstração,Tipo Informação,Período,Codigo Conta,Descr Conta,valConta1,valConta2,valConta3,valConta4,valConta5,valConta6,valConta7,valConta8,valConta9,valConta10,valConta11,valConta12
66,2,2,0,1,Ativo Total,0.0,924465000.0,860473000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67,2,2,0,1.01,Ativo Circulante,0.0,147601000.0,143606000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68,2,2,0,1.01.01,Caixa e Equivalentes de Caixa,0.0,54882000.0,53854000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,2,2,0,1.01.02,Aplicações Financeiras,0.0,5427000.0,4198000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70,2,2,0,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71,2,2,0,1.01.02.01.01,Títulos para Negociação,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,2,2,0,1.01.02.01.02,Títulos Designados a Valor Justo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73,2,2,0,1.01.02.02,Aplicações Financeiras Avaliadas a Valor Justo...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74,2,2,0,1.01.02.03,Aplicações Financeiras Avaliadas ao Custo Amor...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75,2,2,0,1.01.03,Contas a Receber,0.0,17495000.0,22264000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [166]:
# for l in dados_brutos[[True if k[0] == '1' else False for k in dados_brutos['Codigo Conta']]]['Codigo Conta']:
#     print(l)